# Phase 1


## Load Libraries

In [ ]:
!pip install geopandas
import geopandas as gpd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 77.0 MB/s eta 0:00:00


In [648]:
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import math
import re
import datetime
from shapely import wkt

## APIs for pulling in permit files

## Initialize Data Spec

Upload fields.csv and desired permits file for any city

In [790]:
#eventually, the idea will be for permits to change based on the API call and then a city specific dataframe will be created.
#Once all API calls are made, the city specific dataframes will all be concatenated and exported
permits = pd.read_csv('sanfran.csv')

<ipython-input-790-0f2604fa5012>:3: DtypeWarning: Columns (18,22,34,36) have mixed types. Specify dtype option on import or set low_memory=False.
  permits = pd.read_csv('sanfran.csv')


In [791]:
#Initializing the data frame for our final data specification
fields = pd.read_csv('fields.csv', usecols = ['Combined'])
ls_fields = list(fields.Combined.unique())
dataspec = pd.DataFrame(columns = ls_fields, index = [])

## Creating Geo Data Frames

In [792]:
geom_count = 0

for col in permits.columns:
  permits[col] = permits[col].astype(str)
  if permits[col].str.contains('POINT').any():
    geom_count += 1
    geo = col
  else:
    continue

if geom_count > 1:
  print("ERROR")
elif geom_count == 1:
  permits = permits[permits[geo].notna()]
  permits['geometry'] = permits[geo]

permits['geometry'] = permits['geometry'].replace('nan',None)
permits['geometry'] = permits['geometry'].apply(wkt.loads)

In [793]:
permits = gpd.GeoDataFrame(permits, geometry = permits['geometry'], crs=4326)
permits['Longitude'] = permits['geometry'].x
permits['Latitude'] = permits['geometry'].y

# Phase 2.1: Schema Fields 1 - 13

## Unique Land IDs and Unique Building IDs

In [429]:
#Checking for Land IDs
for col in permits.columns:
  # if pd.Series(dataspec['Unique Land Identifier'].values).isnull().all() == True:
  if len(dataspec['Unique Land Identifier'].value_counts()) == 0:
    if re.search(r'(^pin|parcel|pin$)', col, re.IGNORECASE):
      dataspec['Unique Land Identifier'] = pd.Series(permits[col].values)
      break
    elif re.search('folio', col, re.IGNORECASE):
      dataspec['Unique Land Identifier'] = pd.Series(permits[col].values)
      break
    else: continue
  else: continue

In [430]:
#Checking for Land IDs or Building IDs
for col in permits.columns:
  if re.search(r'(^id\b|[^a-zA-Z]id[^a-zA-Z]|\bid$)', col, re.IGNORECASE):
    if len(dataspec[r'Unique Land Identifier'].value_counts()) == 0:
      if len(dataspec[r'Unique Building Identifier'].value_counts()) == 0:
        if re.search(r'build|bin|property', col, re.IGNORECASE):
          dataspec['Unique Building Identifier'] = pd.Series(permits[col].values)
          break
        else:
          dataspec['Unique Land Identifier'] = pd.Series(permits[col].values)
          break
      else: continue
    else: continue
  else: continue

## Address, Street Number, and Street Name



### Street Number

In [431]:
st_num1 = []
st_num2 = []

numsufcount = 0

for col in permits.columns:
  if re.search(r'(address|street|st\.?\b)', col, re.IGNORECASE):
    if not re.search(r'(cont|appli|2|flag|owner)', col, re.IGNORECASE):
      if re.search(r'(no\.?\b|num|number|#|house)', col, re.IGNORECASE):
        if re.search(r'(suffix|suf)', col, re.IGNORECASE):
          numsufcount +=1

for col in permits.columns:
  if re.search(r'(address|street|st)', col, re.IGNORECASE):
    if not re.search(r'cont|appli|2|flag', col, re.IGNORECASE):
      if re.search(r'(no\.?\b|num|number|#|house)', col, re.IGNORECASE):
        if numsufcount != 0:
          if re.search(r'(suffix|suf)', col, re.IGNORECASE):
            st_num2 = pd.Series(permits[col].values).fillna("").astype(str)
        else: continue

In [432]:
num_full = pd.Series()

for col in permits.columns:
  if re.search(r'(address|street|st\.?\b)', col, re.IGNORECASE):
    if not re.search(r'(cont|appli|2|flag|owner)', col, re.IGNORECASE):
      if re.search(r'(no\.?\b|num|number|#|house)', col, re.IGNORECASE):
        if not re.search(r'(suffix|suf)', col, re.IGNORECASE):
          if numsufcount != 0:
            st_num1 = pd.Series(permits[col].values).astype(str)
            num_full = pd.Series(st_num1.str.cat(st_num2, sep=""))
            dataspec['Building Street Number'] = num_full
            break
          elif numsufcount == 0:
            st_num1 = pd.Series(permits[col].values).astype(str)
            num_full = st_num1
            dataspec['Building Street Number'] = num_full
            break

<ipython-input-432-acbba0bfe146>:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  num_full = pd.Series()


### Street Name/Address

In [433]:
#For When there's Direction and Suffix in a Street Name (Which are different columns than the Street's NAME)
st_name1 = [] #bookend 1
st_name2 = [] #The Actual Street Name
st_name3 = [] #bookend 2

dircount = 0 #checking for the existence of 'Direction' in the list of columns: N, E, S, W, North, East, etc.
sufcount = 0 #checking for the existence of 'Suffix' in the list of columns: Ave, Blvd, St, Rd, Street, Avenue, etc.
for col in permits.columns:
  if re.search(r'(address|street|\bst\.?\b)', col, re.IGNORECASE):
    if not re.search(r'cont|appli|2|flag', col, re.IGNORECASE):
      if not re.search(r'(no\.?|num|number|#|house)', col, re.IGNORECASE): #neither direction or suffix counts here have anything to do with street number
        if re.search(r'direction', col, re.IGNORECASE): #may be other variables
          dircount+=1
        elif re.search(r'(suffix|suf)', col, re.IGNORECASE): #may be other variables
          sufcount+=1
  elif col.lower() == "suffix":
    sufcount+=1

for col in permits.columns:
  if re.search(r'(address|street|\bst\.?\b)', col, re.IGNORECASE):
    if not re.search(r'cont|appli|2|flag', col, re.IGNORECASE):
      if not re.search(r'(no\.?|num|number|#|house)', col, re.IGNORECASE):
        if dircount != 0:
          if re.search(r'direction', col, re.IGNORECASE):
            st_name1 = pd.Series(permits[col].values)
          else: continue
        elif sufcount != 0:
          if re.search(r'(suffix|suf)', col, re.IGNORECASE):
            st_name3 = pd.Series(permits[col].values)
          else: continue
  elif col.lower() == "suffix":
    if sufcount != 0: #suffix by itself - most likely street name suffix, not street number suffix
      if re.search(r'(suffix|suf)', col, re.IGNORECASE):
        st_name3 = pd.Series(permits[col].values)
      else: continue

In [434]:
#Concatenating Street Name Related Columns
st_full = pd.Series()

for col in permits.columns:
  if re.search(r'(address|street|\bst\.?\b)', col, re.IGNORECASE):
    if not re.search('cont|appli|2|flag', col, re.IGNORECASE):
      if re.search(r'name', col, re.IGNORECASE):
        if (dircount != 0) & (sufcount != 0):
          st_name2 = pd.Series(permits[col].values)
          st_full = pd.Series(st_name1.str.cat([st_name2, st_name3], sep=" "))
          dataspec['Building Street Name'] = st_full
          break
        elif (dircount != 0) & (sufcount == 0):
          st_name2 = pd.Series(permits[col].values)
          st_full = pd.Series(st_name1.str.cat(st_name2, sep=" "))
          dataspec['Building Street Name'] = st_full
          break
        elif (dircount == 0) & (sufcount != 0):
          st_name2 = pd.Series(permits[col].values)
          st_full = pd.Series(st_name2.str.cat(st_name3, sep=" "))
          dataspec['Building Street Name'] = st_full
          break
        elif (len(dataspec['Building Street Number'].value_counts()) != 0) & ((dircount == 0) & (sufcount == 0)):
          st_name2 = pd.Series(permits[col].values)
          st_full = st_name2
          dataspec['Building Street Name'] = st_full
          break
      elif (len(dataspec['Building Street Number'].value_counts()) == 0) & ((dircount == 0) & (sufcount == 0)):
        dataspec[['Building Street Number', 'Building Street Name']] = pd.Series(permits[col].values).str.split(n=1,expand=True)
        break
      else: continue
    else: continue
  else: continue

<ipython-input-434-a4df76b122e3>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  st_full = pd.Series()


## City, State, and Zip Code

Some cities may not have this information, despite other identifying geographic factors such as latitude/longitude.

In [435]:
for col in permits.columns:
  if re.search('city|boro',col,re.IGNORECASE):
    if not re.search('cont|appli|owner', col, re.IGNORECASE):
      dataspec['Building City'] = pd.Series(permits[col].values)

  if re.search('state', col, re.IGNORECASE):
    if not re.search('id|house|_|lic|con', col, re.IGNORECASE):
      dataspec['Building State'] = pd.Series(permits[col].values)

  if re.search('zip', col, re.IGNORECASE):
    if not re.search('cont|appli|house|2', col, re.IGNORECASE):
      dataspec['Building Zip Code'] = pd.Series(permits[col].values)
      dataspec['Building Zip Code'] = dataspec['Building Zip Code'].fillna(0).astype('int')

  if re.search('country',col,re.IGNORECASE):
    if not re.search('cont|appli|owner', col, re.IGNORECASE):
      dataspec['Building country'] = pd.Series(permits[col].values)

## Building Type/Land-Use (Existing and Proposed)

In [436]:
#Existing Land Use/Building Type - Unfortunately not easily separable
for col in permits.columns:
  if re.search(r'typ|map|use', col, re.IGNORECASE):
    if re.search(r'exist|/^((?!proposed).)*$/', col, re.IGNORECASE): #if the column contains existing|exist|not 'proposed'
      if re.search(r'building|bldg|land', col, re.IGNORECASE):
        if re.search(r'use', col, re.IGNORECASE):
          dataspec['Existing Land Use'] = pd.Series(permits[col].values)
          break
        elif re.search(r'typ', col, re.IGNORECASE):
          dataspec['Existing Land Use'] = pd.Series(permits[col].values)
          break
      elif re.search(r'use', col, re.IGNORECASE):
        dataspec['Existing Land Use'] = pd.Series(permits[col].values)
        break
      elif re.search(r'typ', col, re.IGNORECASE):
        dataspec['Existing Land Use'] = pd.Series(permits[col].values)
        break
      elif re.search(r'map', col, re.IGNORECASE):
        dataspec['Existing Land Use'] = pd.Series(permits[col].values)
        break

for col in permits.columns:
  if len(dataspec['Existing Land Use'].value_counts()) != 0:
    break
  else:
    if re.search(r'typ|map|use', col, re.IGNORECASE):
      if re.search(r'exist|/^((?!proposed).)*$/', col, re.IGNORECASE):
        if re.search(r'occupancy', col, re.IGNORECASE):
          dataspec['Existing Land Use'] = pd.Series(permits[col].values)
          break

In [437]:
#Proposed Land Use/Building Type - Unfortunately not easily separable
for col in permits.columns:
  if re.search(r'typ|map|use', col, re.IGNORECASE):
    if re.search(r'proposed|/^((?!exist).)*$/', col, re.IGNORECASE):
      if re.search(r'building|bldg|land', col, re.IGNORECASE):
        if re.search(r'use', col, re.IGNORECASE):
          dataspec['Proposed Land Use'] = pd.Series(permits[col].values)
          break
        elif re.search(r'typ', col, re.IGNORECASE):
          dataspec['Proposed Land Use'] = pd.Series(permits[col].values)
          break
      elif re.search(r'use', col, re.IGNORECASE):
        dataspec['Proposed Land Use'] = pd.Series(permits[col].values)
        break
      elif re.search(r'typ', col, re.IGNORECASE):
        dataspec['Proposed Land Use'] = pd.Series(permits[col].values)
        break
      elif re.search(r'map', col, re.IGNORECASE):
        dataspec['Proposed Land Use'] = pd.Series(permits[col].values)
        break
    elif len(dataspec['Proposed Land Use'].value_counts()) == 0:
      if re.search(r'use', col, re.IGNORECASE):
        dataspec['Proposed Land Use'] = pd.Series(permits[col].values)
        break
      elif re.search(r'map', col, re.IGNORECASE):
        dataspec['Proposed Land Use'] = pd.Series(permits[col].values)
        break

for col in permits.columns:
  if len(dataspec['Proposed Land Use'].value_counts()) != 0:
    break
  else:
    if re.search(r'typ|map|use', col, re.IGNORECASE):
      if re.search(r'proposed|/^((?!exist).)*$/', col, re.IGNORECASE):
        if re.search(r'occupancy', col, re.IGNORECASE):
          dataspec['Proposed Land Use'] = pd.Series(permits[col].values)
          break

## Zoning Designation (Existing and Proposed)

In [438]:
#Existing and Proposed Zoning Designations
for col in permits.columns:
  if re.search(r'zoning|zon', col, re.IGNORECASE):
    if re.search(r'exist|/^((?!proposed).)*$/', col, re.IGNORECASE):
      if re.search(r'use|typ', col, re.IGNORECASE):
        if re.search(r'use', col, re.IGNORECASE):
          dataspec['Existing Zoning Designation'] = pd.Series(permits[col].values)
          break
        if re.search(r'typ', col, re.IGNORECASE):
          dataspec['Existing Zoning Designation'] = pd.Series(permits[col].values)
          break

for col in permits.columns:
  if re.search(r'zoning|zon', col, re.IGNORECASE):
    if re.search(r'proposed|/^((?!exist).)*$/', col, re.IGNORECASE):
      if re.search(r'use|typ', col, re.IGNORECASE):
        if re.search(r'use', col, re.IGNORECASE):
          dataspec['Proposed Zoning Designation'] = pd.Series(permits[col].values)
          break
        if re.search(r'typ', col, re.IGNORECASE):
          dataspec['Proposed Zoning Designation'] = pd.Series(permits[col].values)
          break

# Phase 2.1: Testing

In [439]:
dataspec.iloc[:10,:13]

,Unique Building Identifier,Unique Land Identifier,Building Street Number,Building Street Name,Building City,Building State,Building Zip Code,Building Country,Building Type,Existing Land Use,Proposed Land Use,Existing Zoning Designation,Proposed Zoning Designation
0,NaN,316100702,6406,OLD BURLESON RD UNIT 160,AUSTIN,TX,20,NaN,NaN,NaN,Commercial,NaN,NaN
1,NaN,234030419,6801,BURNET RD,AUSTIN,TX,15,NaN,NaN,NaN,Commercial,NaN,NaN
2,NaN,218090127,1037,E 43RD ST,AUSTIN,TX,63,NaN,NaN,NaN,Residential,NaN,NaN
3,NaN,308160701,1430,FRONTIER VALLEY DR UNIT 38,AUSTIN,TX,52,NaN,NaN,NaN,Residential,NaN,NaN
4,NaN,416180307,1911,BISSEL LN,AUSTIN,TX,23,NaN,NaN,NaN,Residential,NaN,NaN
5,NaN,236191001,1006,WISTERIA CIR,AUSTIN,TX,76,NaN,NaN,NaN,Residential,NaN,NaN
6,NaN,234030419,6801,BURNET RD,AUSTIN,TX,15,NaN,NaN,NaN,Commercial,NaN,NaN
7,NaN,231110607,7006,MIRANDA DR,AUSTIN,TX,17,NaN,NaN,NaN,Residential,NaN,NaN
8,NaN,254430237,13625,COOMER PATH,AUSTIN,TX,39,NaN,NaN,NaN,Residential,NaN,NaN
9,NaN,412440605,8605,DARK SHADOW CV,AUSTIN,TX,51,NaN,NaN,NaN,Residential,NaN,NaN


# Phase 2.2: Schema Fields 14 - 26

## Permit Number

In [554]:
for col in permits.columns:
  if re.search('permit', col, re.IGNORECASE):
    if re.search(r'(no\.?\b|num|number|#)', col, re.IGNORECASE):
      if not re.search('master', col, re.IGNORECASE):
        dataspec['Permit Number'] = pd.Series(permits[col].values)
      else: continue
    else: continue
  else: continue

## Permit Type/Sub-Type, Work Type/Sub-Type, Work Description

In [555]:
for col in permits.columns:
  if re.search(r'permit', col, re.IGNORECASE):
    if re.search(r'typ|class', col, re.IGNORECASE): #searching for permit type
      if not re.search(r'definit|descr|work', col, re.IGNORECASE): #will relegate definition/description elsewhere
        if not re.search(r'sub', col, re.IGNORECASE):
          dataspec['Permit Type'] = pd.Series(permits[col].values)
        else:
          dataspec['Permit Sub-Type'] = pd.Series(permits[col].values)
      else:
        if not re.search(r'work', col, re.IGNORECASE):
          if len(dataspec['Permit Type'].value_counts()) == 0:
            dataspec['Permit Type'] = pd.Series(permits[col].values)
          elif len(dataspec['Permit Sub-Type'].value_counts()) == 0:
            dataspec['Permit Sub-Type'] = pd.Series(permits[col].values)
          else: continue
        else: continue
    else: continue

In [556]:
for col in permits.columns:
  if re.search(r'work|action', col, re.IGNORECASE):
    if re.search(r'typ', col, re.IGNORECASE): #searching for permit type
      if not re.search(r'definit|descr', col, re.IGNORECASE):
        if not re.search(r'sub', col, re.IGNORECASE):
          dataspec['Work Type'] = pd.Series(permits[col].values)
        else:
          dataspec['Work Sub-Type'] = pd.Series(permits[col].values)
      else:
        if len(dataspec['Work Type'].value_counts()) == 0:
          dataspec['Work Type'] = pd.Series(permits[col].values)
        elif len(dataspec['Work Sub-Type'].value_counts()) == 0:
          dataspec['Work Sub-Type'] = pd.Series(permits[col].values)
    else: continue
  else: continue

In [557]:
for col in permits.columns:
  if re.search(r'definit|descr', col, re.IGNORECASE):
    if not re.search(r'permit|work', col, re.IGNORECASE):
      dataspec['Project Description'] = pd.Series(permits[col].values)
      break #finds the first column that mentions description without 'permit' or 'work' --> might need more rules
    else: continue
  else: continue

## Dates: Permits, Permit Status, Certificate of Occupancy

Completed Date may mean very different things depending on the city. It is assumed NOT to mean: Permit Expiration Date or Certificate of Occupancy Issued Date.

For our purposes, we will assume Completed Date means Inspection Completion Date

In [562]:
for col in permits.columns:
  if re.search(r'date', col, re.IGNORECASE): #focus first on 'permits' related dates, then other dates
    if re.search(r'appli|creat', col, re.IGNORECASE): #capturing Application and Creation Date fields
      print(col)
      permits[col] = pd.to_datetime(permits[col])
      dataspec['Permit Application Date'] = pd.Series(permits[col].values)
    elif re.search(r'approv', col, re.IGNORECASE):
      permits[col] = pd.to_datetime(permits[col])
      dataspec['Permit Approval Date'] = pd.Series(permits[col].values)
    elif re.search(r'issu', col, re.IGNORECASE):
      permits[col] = pd.to_datetime(permits[col])
      dataspec['Permit Issued Date'] = pd.Series(permits[col].values)
    elif re.search(r'expir', col, re.IGNORECASE):
      permits[col] = pd.to_datetime(permits[col])
      dataspec['Permit Expiration Date'] = pd.Series(permits[col].values)
    elif re.search(r'comple', col, re.IGNORECASE):
      permits[col] = pd.to_datetime(permits[col])
      dataspec['Inspection Completion Date'] = pd.Series(permits[col].values)
    else: #check for other variations, but avoid connecting 'certificate of occupancy', 'construction', 'status' dates, etc.
      continue

if (len(dataspec['Permit Approval Date'].value_counts()) == 0) & (len(dataspec['Permit Issued Date'].value_counts()) != 0):
        dataspec['Permit Approval Date'] = dataspec['Permit Issued Date']

Permit Creation Date


In [559]:
for col in permits.columns:
  if re.search(r'status', col, re.IGNORECASE):
    if re.search(r'date', col, re.IGNORECASE):
      dataspec['Permit Current Status Date'] = pd.Series(permits[col].values)
    else:
      dataspec['Permit Current Status'] = pd.Series(permits[col].values)
  else: continue

In [560]:
for col in permits.columns:
  if re.search(r'date', col, re.IGNORECASE):
    if re.search(r'cert|occup', col, re.IGNORECASE):
      dataspec['Certificate of Occupancy Issued Date'] = pd.Series(permits[col].values)

# Phase 2.2: Testing

In [561]:
dataspec.iloc[:10,13:26]

,Permit Number,Permit Application Date,Permit Approval Date,Permit Issued Date,Permit Expiration Date,Certificate of Occupancy Issued Date,Permit Type,Permit Sub-Type,Work Type,Work Sub-Type,Project Description,Permit Current Status,Permit Current Status Date
0,M1179687,2020-01-02,2020-01-02 08:01:00,2020-01-02 08:01:00,NaN,NaN,8,otc alterations permit,NaN,NaN,street space,issued,1/2/2020 8:01
1,2.02001E+11,2020-01-02,2020-01-02 13:25:00,2020-01-02 13:25:00,NaN,NaN,8,otc alterations permit,NaN,NaN,"lower lvl: sheet rock wall and closet, refinis...",issued,1/2/2020 13:25
2,2.02001E+11,2020-01-02,2020-01-02 09:02:00,2020-01-02 09:02:00,NaN,NaN,8,otc alterations permit,NaN,NaN,ref pa# 201901170631; suite 01-51: fire sprink...,issued,1/2/2020 9:02
3,M1179707,2020-01-02,2020-01-02 08:19:00,2020-01-02 08:19:00,NaN,NaN,8,otc alterations permit,NaN,NaN,street space,issued,1/2/2020 8:19
4,M1179707,2020-01-02,2020-01-02 08:19:00,2020-01-02 08:19:00,NaN,NaN,8,otc alterations permit,NaN,NaN,street space,issued,1/2/2020 8:19
5,2.02001E+11,2020-01-02,2020-01-08 14:48:00,2020-01-08 14:48:00,NaN,NaN,8,otc alterations permit,NaN,NaN,to comply w/ physical inspection report # cc-8302,complete,1/10/2020 10:59
6,2.02001E+11,2020-01-02,2020-01-08 14:48:00,2020-01-08 14:48:00,NaN,NaN,8,otc alterations permit,NaN,NaN,to comply w/ physical inspection report # cc-8302,complete,1/10/2020 10:59
7,2.02001E+11,2020-01-02,2020-01-03 10:24:00,2020-01-03 10:24:00,NaN,NaN,8,otc alterations permit,NaN,NaN,revision to 201904309293 structural changes pe...,complete,1/13/2021 10:27
8,M1179727,2020-01-02,2020-01-02 08:41:00,2020-01-02 08:41:00,NaN,NaN,8,otc alterations permit,NaN,NaN,street space,issued,1/2/2020 8:41
9,2.02001E+11,2020-01-02,2020-01-14 09:18:00,2020-01-14 09:18:00,NaN,NaN,8,otc alterations permit,NaN,NaN,like for like replacement of 3 ac units,issued,1/14/2020 9:18


# Phase 2.3: Schema Fields 28 - 40

## Floors, Floor/Roof Rail Height, Building Height

Assumptions: Floor Height = 10 feet, Rooftop Rail Height = 4 ft

In [516]:
for col in permits.columns:
  if re.search(r'floor|\bstor', col, re.IGNORECASE):
    if re.search(r'(no\.?\b|num|number|#)', col, re.IGNORECASE):
      if re.search(r'exist|/^((?!proposed).)*$/', col, re.IGNORECASE):
        dataspec['Existing Number of Floors Above Grade'] = pd.Series(permits[col].values)
        break
      else: continue
    else: continue
  else: continue

for col in permits.columns:
  if re.search(r'floor|\bstor', col, re.IGNORECASE):
    if re.search(r'(no\.?\b|num|number|#)', col, re.IGNORECASE):
      if re.search(r'proposed|/^((?!exist).)*$/', col, re.IGNORECASE):
        dataspec['Proposed Number of Floors Above Grade'] = pd.Series(permits[col].values)
        break
      else:
        if len(dataspec['Existing Number of Floors Above Grade'].value_counts()) == 0:
          dataspec['Existing Number of Floors Above Grade'] = pd.Series(permits[col].values)
          break
        else: continue
    else: continue
  else: continue

In [517]:
dataspec.loc[dataspec['Existing Number of Floors Above Grade'].notnull(), 'Existing Building Height'] = (dataspec['Existing Number of Floors Above Grade']*10)+4
dataspec.loc[dataspec['Proposed Number of Floors Above Grade'].notnull(), 'Proposed Building Height'] = (dataspec['Proposed Number of Floors Above Grade']*10)+4

## Geometries: Latitude, Longitude, Footprint (as applicable)

In [794]:
for col in permits.columns:
  if re.search(r'latitude|point', col, re.IGNORECASE):
    if re.search(r'latitude|y', col, re.IGNORECASE):
      dataspec['Latitude'] = pd.Series(permits[col].values)

for col in permits.columns:
  if re.search(r'longitude|point', col, re.IGNORECASE):
    if re.search(r'longitude|x', col, re.IGNORECASE):
      dataspec['Longitude'] = pd.Series(permits[col].values)

# Phase 2.3: Testing

In [795]:
dataspec.iloc[:10,26:38]

,Existing Number of Floors Above Grade,Proposed Number of Floors Above Grade,Number of Floors Below Grade,Floor Height,Existing Building Height,Proposed Building Height,Gross Floor Area,Inspection Type,Inspection Completion Date,Latitude,Longitude,Building Footprint
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.802432,-122.425800,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.762565,-122.406900,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.789855,-122.396801,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.745012,-122.427511,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.745012,-122.427511,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.767121,-122.427480,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.767121,-122.427480,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.751548,-122.440827,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.757409,-122.418619,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.781681,-122.404102,NaN


# Archive

In [ ]:
for col in permits.columns:



    if re.search('construction|bldg|commercial',col,re.IGNORECASE):
        if not re.search('sqft|date|exist|pro',col,re.IGNORECASE):
            dataspec['Building Type'] = pd.Series(permits[col].values)

    if re.search('occupancy|class',col,re.IGNORECASE):
        if re.search('type',col,re.IGNORECASE):
            dataspec['Building Type'] = pd.Series(permits[col].values)

    if re.search('class',col,re.IGNORECASE):
        if re.search('mapped',col,re.IGNORECASE):
             dataspec['Building Type'] = pd.Series(permits[col].values)

    if re.search('group|proposed',col,re.IGNORECASE):
        if not re.search('code|des|stor|unit|use|con',col,re.IGNORECASE):
            dataspec['Land-Use'] = pd.Series(permits[col].values)

    if re.search('occupancy',col,re.IGNORECASE):
        if re.search('type',col,re.IGNORECASE):
            dataspec['Land-Use'] = pd.Series(permits[col].values)

###########################################################################

    if re.search('num', col, re.IGNORECASE):
        if re.search('permit|ivr', col, re.IGNORECASE):
            if not re.search('master', col, re.IGNORECASE):
                dataspec['Permit Number'] = pd.Series(permits[col].values)

    if col == 'PERMIT#':
        dataspec['Permit Number'] = pd.Series(permits[col].values)

    if col == 'Job #':
        dataspec['Permit Number'] = pd.Series(permits[col].values)

    if re.search('date', col, re.IGNORECASE):
        if re.search('appli|start|creation', col, re.IGNORECASE):
            dataspec['Permit Application Date'] = pd.Series(permits[col].values)

    if re.search('set', col, re.IGNORECASE):
        if not re.search('photovo|plan', col, re.IGNORECASE):
            dataspec['Permit Application Date'] = pd.Series(permits[col].values)

    if re.search('issu', col, re.IGNORECASE):
        if re.search('date', col, re.IGNORECASE):
            if not re.search('coi', col, re.IGNORECASE):
                dataspec['Permit Issued Date'] = pd.Series(permits[col].values)

    if col == 'issued':
        dataspec['Permit Issued Date'] = pd.Series(permits[col].values)

    if re.search('date', col, re.IGNORECASE):
        if re.search('completed|expir', col, re.IGNORECASE):
            if not re.search('valuation', col, re.IGNORECASE):
                dataspec['Permit Expiration Date'] = pd.Series(permits[col].values)

    if re.search('final', col, re.IGNORECASE):
        if not re.search('valuation', col, re.IGNORECASE):
            dataspec['Permit Expiration Date'] = pd.Series(permits[col].values)
###########################################################################

    if re.search('type', col, re.IGNORECASE):
        if re.search('permit', col, re.IGNORECASE):
            if not re.search('work|sub|lic|desc|map|def', col, re.IGNORECASE):
                dataspec['Permit Type'] = pd.Series(permits[col].values)

    if col == 'type':
        dataspec['Permit Type'] = pd.Series(permits[col].values)

    if re.search('type', col, re.IGNORECASE):
        if re.search('sub|work|action', col, re.IGNORECASE):
            if not re.search('permit', col, re.IGNORECASE):
                dataspec['Work Type'] = pd.Series(permits[col].values)

    if re.search('type', col, re.IGNORECASE):
        if re.search('permit', col, re.IGNORECASE):
            if re.search('def', col, re.IGNORECASE):
                dataspec['Work Type'] = pd.Series(permits[col].values)

    if col == 'work':
        dataspec['Work Type'] = pd.Series(permits[col].values)

    if re.search('class|scope', col, re.IGNORECASE):
        if re.search('work', col, re.IGNORECASE):
            if not re.search('map', col, re.IGNORECASE):
                dataspec['Work Type'] = pd.Series(permits[col].values)

    if re.search('status', col, re.IGNORECASE):
        if not re.search('filing|date', col, re.IGNORECASE):
            dataspec['Permit Current Status'] = pd.Series(permits[col].values)

    if re.search('status', col, re.IGNORECASE):
        if re.search('date', col, re.IGNORECASE):
            dataspec['Permit Current Status Date'] = pd.Series(permits[col].values)

    if re.search('number', col, re.IGNORECASE):
        if re.search('floor|stories', col, re.IGNORECASE):
            if not re.search('pro', col, re.IGNORECASE):
                dataspec['Number of Floors Above Grade'] = pd.Series(permits[col].values)

    if re.search('y_web|lat|pointy', col, re.IGNORECASE):
        if not re.search('soc|mup', col, re.IGNORECASE):
            dataspec['Latitude'] = pd.Series(permits[col].values)

    if re.search('x_web|long|pointx', col, re.IGNORECASE):
        if not re.search('soc', col, re.IGNORECASE):
            dataspec['Longitude'] = pd.Series(permits[col].values)

#####################################################################
